# InPars - Finetuning

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [1]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-9"

## Libraries installation

In [2]:
!pip install transformers -q

In [3]:
!pip install jsonlines -q

In [4]:
!pip install evaluate -q

In [5]:
!pip install trectools -q

In [11]:
import random
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from statistics import mean, stdev

Random seeds definition, to enable replication of results.

In [7]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Dataset processing

In [ ]:
dataset_file = f"{main_dir}/trec-covid/monique_monteiro_1000_queries.jsonl"

In [ ]:
!head {dataset_file}

{"query": "What is the most suitable protein for a diagnostic approach for Salmonella Enteritidis and why?", "positive_doc_id": "m1cmkkw3", "negative_doc_ids": ["0o3mryu1", "qpq7i1ya", "j5mkparg", "auoo0dm5", "dqfvrerw"]}
{"query": "What is Cryptosporidium parvum and why is it a major cause of disease in both humans and animals?", "positive_doc_id": "ukbl0svm", "negative_doc_ids": ["k3u2nvpe", "gc11fyms", "xoq9qblv", "20o4ufa3", "3huo5nf0"]}
{"query": "What is the role of the renin-angiotensin-aldosterone system in the context of SARS-CoV-2 infection?", "positive_doc_id": "12o4zey2", "negative_doc_ids": ["6gd6nwpu", "dt4t2wos", "8zwfkken", "sv7xpi4f", "6pf73z08"]}
{"query": "What are the functions of individual endolysosomal proteases in cellular processes such as autophagy and lipoprotein particle degradation?", "positive_doc_id": "eqv6a7tj", "negative_doc_ids": ["gmrty2uu", "uzn214j6", "032utjfh", "efet3ozc", "0muwl6oc"]}
{"query": "What is the prevalence of olfactory dysfunction in 

In [8]:
import jsonlines

id_to_text = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    text = item["title"] + ' ' + item["text"]
    id_to_text[id] = text

In [9]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

def generate_training_data(dataset_file, model_name):
  dataset = []
  dataset_ids = []
  i = 0

  with open(dataset_file, 'r') as f:
    for line in f:
      data = json.loads(line)
      query = data["query"]
      positive_doc_id = data["positive_doc_id"]
      negative_doc_ids = data["negative_doc_ids"]
      
      #Chooses a random negative document
      negative_doc_id = random.choice(negative_doc_ids)

      #Gets the documents texts
      positive_doc = id_to_text[positive_doc_id]
      negative_doc = id_to_text[negative_doc_id]

      dataset.append((query, positive_doc, negative_doc))
      dataset_ids.append((i, positive_doc_id, negative_doc_id))

      i+=1

  df = pd.DataFrame(dataset, columns=['query', 'pos', 'neg'])

  df_pos = pd.DataFrame()
  df_neg = pd.DataFrame()

  for index, row in df.iterrows():
    if model_name == 'microsoft/MiniLM-L12-H384-uncased':
      df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1.0}, 
                             ignore_index=True)
      df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0.0}, 
                             ignore_index=True)
    elif model_name == 'cross-encoder/ms-marco-MiniLM-L-6-v2':
      df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":True}, 
                             ignore_index=True)
      df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":False}, 
                             ignore_index=True)

  print(model_name)
  print(df_pos.head())
  X_train_pos = df_pos.drop("score", axis=1)
  Y_train_pos = df_pos["score"]

  X_train_pos, X_val_pos, Y_train_pos, Y_val_pos = train_test_split(X_train_pos, 
                                                                    Y_train_pos, 
                                                                    test_size=0.1, 
                                                                    random_state=42)

  X_train_neg = df_neg.drop("score", axis=1)
  Y_train_neg = df_neg["score"]

  X_train_neg, X_val_neg, Y_train_neg, Y_val_neg = train_test_split(X_train_neg, 
                                                                    Y_train_neg, 
                                                                    test_size=0.1, 
                                                                    random_state=42)

  X_train = pd.concat([X_train_pos, X_train_neg], axis=0, ignore_index=True)
  Y_train = pd.concat([Y_train_pos, Y_train_neg], axis=0, ignore_index=True)
  X_val = pd.concat([X_val_pos, X_val_neg], axis=0, ignore_index=True)
  Y_val = pd.concat([Y_val_pos, Y_val_neg], axis=0, ignore_index=True)

  return X_train, Y_train, X_val, Y_val


## Finetuning with microsoft/MiniLM-L12-H384-uncased

Data preparation, with tokenization and dataset/dataloaders construction.

In [ ]:
model_name = 'microsoft/MiniLM-L12-H384-uncased'

In [ ]:
X_train, Y_train, X_val, Y_val = generate_training_data(dataset_file, model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:


lengths = [len(tokens) for tokens in tokenizer(list(X_train["passage"])[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

In [12]:
max_length = 512
max_length_query = int(max_length/4)
max_length_passage = int(3*max_length/4)
max_length_query, max_length_passage

(128, 384)

In [ ]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

train_queries_tokenized = tokenizer(train_queries, truncation=True, max_length=max_length_query)
train_passages_tokenized = tokenizer(train_passages, truncation=True, max_length=max_length_passage)
val_queries_tokenized = tokenizer(val_queries, truncation=True, max_length=max_length_query)
val_passages_tokenized = tokenizer(val_passages, truncation=True, max_length=max_length_passage)

In [13]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, queries, passages, targets):
        self.queries = queries
        self.passages = passages
        self.targets = targets
    
    def __len__(self):
        return len(self.queries['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.queries['input_ids'][idx] + self.passages['input_ids'][idx],
            'attention_mask': self.queries['attention_mask'][idx] + self.passages['attention_mask'][idx],
            #'labels': float(self.targets[idx])
            'labels': int(self.targets[idx])
        }


In [ ]:
dataset_train = Dataset(train_queries_tokenized, train_passages_tokenized, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0

In [ ]:
dataset_valid = Dataset(val_queries_tokenized, val_passages_tokenized, Y_val)

In [14]:
from transformers import BatchEncoding

# Tokens do tipo "pad" para textos com tamanho inferior ao máximo suportado.
def collate_fn(batch):
  return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))


In [ ]:
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


### Training Loop

In [15]:
from tqdm import tqdm 

def evaluate(model, dataloader, set_name, model_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            
            if model_name == 'cross-encoder/ms-marco-MiniLM-L-6-v2':
              preds  = (outputs.logits.view(-1)>0).float()
            
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')
    return correct / len(dataloader.dataset)
     

In [16]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
from transformers.optimization import get_constant_schedule
import os
import shutil

def train(model_name, epochs = 5, lr=5e-5):
  model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
  print('Parameters', model.num_parameters())

  optimizer = optim.AdamW(model.parameters(), lr)
  num_training_steps = epochs * len(dataloader_train)

  num_warmup_steps = int(num_training_steps * 0.1)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, 
                                              num_training_steps)
  #scheduler = get_constant_schedule(optimizer)

  evaluate(model=model, dataloader=dataloader_valid, set_name='Valid', 
           model_name=model_name)
  best_acc = 0

  # Training loop
  for epoch in tqdm(range(epochs), desc='Epochs'):
      model.train()
      train_losses = []
      for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', 
                        disable=False):
          optimizer.zero_grad()
          outputs = model(**batch.to(device))
          loss = outputs.loss
          loss.backward()
          optimizer.step()
          scheduler.step()
          train_losses.append(loss.cpu().item())

      print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
      acc = evaluate(model=model, dataloader=dataloader_valid, set_name='Valid', 
                     model_name=model_name)
      #Saves the best checkpoint
      if acc > best_acc:
        best_acc = acc
        if os.path.exists(f'{MODELS_PATH}/best_checkpoint'):
          shutil.rmtree(f'{MODELS_PATH}/best_checkpoint')
        model.save_pretrained(f'{MODELS_PATH}/best_checkpoint')
  
  return model

In [18]:
MODELS_PATH = '/content/gdrive/MyDrive/Unicamp-aula-9'

In [ ]:
model = train(model_name, 5)
model_name = model_name.replace('/','_')
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/7 [00:00<?, ?it/s]

Valid loss: 0.697; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/57 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.02


Valid:   0%|          | 0/7 [00:00<?, ?it/s]

Valid loss: 0.077; Valid accuracy: 0.980


('/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-9/tokenizer_ranker/tokenizer.json')

### Reranking

Evaluation on TREC-COVID

In [ ]:
id_to_query = dict()

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    text = item["text"]
    id_to_query[id] = text

In [ ]:
import jsonlines

id_to_doc = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    text = item["title"] + ' ' + item["text"]
    id_to_doc[id] = text

In [ ]:
!head {main_dir}/trec-covid/run.trec-covid.bm25tuned.txt

1 Q0 dv9m19yk 1 4.158100 Anserini
1 Q0 kgifmjvb 2 3.338900 Anserini
1 Q0 wmfcey6f 3 3.338899 Anserini
1 Q0 safr9z37 4 3.220100 Anserini
1 Q0 0paafp5j 5 3.207300 Anserini
1 Q0 96zsd27n 6 3.207299 Anserini
1 Q0 4dtk1kyh 7 3.184800 Anserini
1 Q0 lhd0jn0z 8 2.903200 Anserini
1 Q0 55dihml5 9 2.899800 Anserini
1 Q0 qtx0d5f8 10 2.888800 Anserini


In [181]:
import pickle
import os

def tokenize_test_queries_and_passages():
  tokenized_queries = None
  tokenized_passages = None

  if os.path.exists(f"{main_dir}/trec-covid/tok_queries_test.pickle"):
    with open(f"{main_dir}/trec-covid/tok_queries_test.pickle", "rb") as f:
      print("Loading test queries...")
      tokenized_queries = pickle.load(f) 

  if os.path.exists(f"{main_dir}/trec-covid/tok_passages_test.pickle"):
    with open(f"{main_dir}/trec-covid/tok_passages_test.pickle", "rb") as f:
      print("Loading test passages...")
      tokenized_passages = pickle.load(f) 

  query_ids = []
  queries = []
  passage_ids = []
  passages = []

  with open(f'{main_dir}/trec-covid/run.trec-covid.bm25tuned.txt') as f:
    for line in f:
        fields = line.strip().split()
        query_id = fields[0]
        query_ids.append(query_id)
        passage_id = fields[2]
        passage_ids.append(passage_id)
        
        if not tokenized_queries:
          query_text = id_to_query[query_id]
          queries.append(query_text)

        if not tokenized_passages:
          passage_text = id_to_doc[passage_id]
          passages.append(passage_text)

  if not tokenized_queries:
    tokenized_queries = tokenizer(queries, max_length=max_length_query, truncation=True)

    with open(f"{main_dir}/trec-covid/tok_queries_test.pickle", 'wb') as f:
      pickle.dump(tokenized_queries, f)

  if not tokenized_passages:
    tokenized_passages = tokenizer(passages, max_length=max_length_passage, truncation=True)

    with open(f"{main_dir}/trec-covid/tok_passages_test.pickle", 'wb') as f:
      pickle.dump(tokenized_passages, f)

  return tokenized_queries, tokenized_passages

In [ ]:
tokenized_queries, tokenized_passages = tokenize_test_queries_and_passages()
dataset_test = Dataset(tokenized_queries, tokenized_passages, [1]*len(tokenized_queries['input_ids']))

In [ ]:
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:

def evaluate_test_dataset(model, dataloader, set_name, use_logits=False):
    scores = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            if use_logits:
              # Usa os logits brutos
              pos_score = outputs.logits[:,1]
            else:
              # Usa os logits normalizados pelo softmax (por default)
              pos_score = torch.softmax(outputs.logits,1)[:,1]
            scores = scores + pos_score.tolist()
    return scores

In [162]:
def evaluate_ndcg_10(scores, model_name, eval_desc):
  zipped_results = []

  #Por alguma razão misteriosa, o zip do Python não funcionou, deixou a lista 
  #vazia ou impossível de ser iterada.
  for i, query_id in enumerate(query_ids):
    zipped_results.append((query_id, passage_ids[i], scores[i]))

  #Quebra a lista em sublistas por query
  prev_query_id = -1
  sublists = []
  current_list = []

  for query_id, passage_id, score in zipped_results:
    if query_id != prev_query_id:
      if len(current_list) > 0:
        sublists.append(current_list)
        current_list = []
    current_list.append((query_id, passage_id, score))
    prev_query_id = query_id

  if len(current_list) > 0:
    sublists.append(current_list)

  # Ordena cada sublista
  sorted_list = []

  for sublist in sublists:
    sorted_sublist = sorted(sublist, key=lambda x: x[2], reverse=True)
    sorted_list += sorted_sublist

  # Gera o arquivo de run no formato TREC
  trec_run_file = f"{main_dir}/trec-covid/run.trec-covid.bert_reranked_{model_name.replace('/', '_')}_{eval_desc}.trec"
  with open(trec_run_file, "w") as f:
    for i, (query_id, passage_id, score) in enumerate(sorted_list):
      f.write(f'{query_id}\t{passage_id}\t{i+1}\t{score}\tbert_reranked_{model_name}\n')

  return trec_run_file

In [ ]:

softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)

Test:   0%|          | 0/1563 [00:00<?, ?it/s]

Test:   0%|          | 0/1563 [00:00<?, ?it/s]

In [ ]:

trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [ ]:
!head {trec_file_softmax}

1	ews32six	1	0.9927167296409607	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	k4l70wf6	2	0.9926959276199341	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	fraczoxu	3	0.9926958084106445	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	zpb9jicw	4	0.9926846027374268	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	dt0b87me	5	0.9926679730415344	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	8f5m0gej	6	0.9926672577857971	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	67fyu0i5	7	0.9926584362983704	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	o7kl258h	8	0.9926577210426331	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	1ew0p6x7	9	0.9926537275314331	bert_reranked_microsoft_MiniLM-L12-H384-uncased
1	2cwvga0k	10	0.9926504492759705	bert_reranked_microsoft_MiniLM-L12-H384-uncased


In [ ]:
import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [ ]:
!head {main_dir}/trec-covid/test.tsv

query-id	corpus-id	score
1	005b2j4b	2
1	00fmeepz	1
1	g7dhmyyo	2
1	0194oljo	1
1	021q9884	1
1	02f0opkr	1
1	047xpt2c	0
1	04ftw7k9	0
1	pl9ht0d0	0


In [ ]:
from evaluate import load

def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10'] 

In [ ]:
import pandas as pd

run_trec_file_softmax = pd.read_csv(trec_file_softmax, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_softmax["q0"] = "q0"
run_trec_file_softmax = run_trec_file_softmax.to_dict(orient="list")

In [ ]:
import pandas as pd

run_trec_file_logits = pd.read_csv(trec_file_logits, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_logits["q0"] = "q0"
run_trec_file_logits = run_trec_file_logits.to_dict(orient="list")

In [ ]:
eval_ndcg10(run_trec_file_softmax)

0.6110263574494792

In [ ]:
eval_ndcg10(run_trec_file_logits)

0.6125945845401235

## Finetuning with cross-encoder/ms-marco-MiniLM-L-6-v2

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils import data
from transformers import BatchEncoding

In [ ]:
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:


class MSMARCODataset(data.Dataset):
    def __init__(self, tokenizer, query, passages, targets, max_lenght=356):
        self.tokenizer = tokenizer
        self.query = query
        self.passages = passages
        self.targets = targets
        self.max_lenght = max_lenght
    
    def __len__(self):
        return len(self.query)

    def __getitem__(self, idx):
        instruction_token = self.tokenizer(self.query[idx],self.passages[idx],
                                           max_length=self.max_lenght, 
                                           truncation=True,
                                           padding="max_length", 
                                           return_tensors='pt')


        return {'input_ids':torch.squeeze(instruction_token['input_ids']).long().to(device),\
               'attention_mask':torch.squeeze(instruction_token['attention_mask']).long().to(device), \
               'labels':torch.tensor(self.targets[idx], dtype=torch.float16)}
              
def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
    

In [ ]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset_train = MSMARCODataset(tokenizer, train_queries, train_passages, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0

In [ ]:
dataset_val = MSMARCODataset(tokenizer, val_queries, val_passages, Y_val)
assert len(dataset_val[0]['input_ids']) > 0
assert len(dataset_val[1]['attention_mask']) > 0

In [ ]:
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_val, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [ ]:
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"


In [ ]:
model = train(model_name, 40, lr=1e-3)

Parameters 22713601


Valid: 100%|██████████| 7/7 [00:01<00:00,  4.64it/s]


Valid loss: 35.093; Valid accuracy: 0.510


Train: 100%|██████████| 57/57 [00:27<00:00,  2.07it/s]


Epoch: 1 Training loss: 6.17


...

Valid loss: 0.040; Valid accuracy: 0.715



Train: 100%|██████████| 57/57 [00:27<00:00,  2.09it/s]


Epoch: 40 Training loss: 0.00



Epochs: 100%|██████████| 40/40 [20:18<00:00, 30.46s/it]

Valid loss: 0.039; Valid accuracy: 0.765


In [ ]:
 model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/best_checkpoint').to(device)

In [ ]:
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Valid: 100%|██████████| 7/7 [00:01<00:00,  4.47it/s]

Valid loss: 0.137; Valid accuracy: 0.865


0.865

In [ ]:
import pickle
import os

query_ids = []
queries = []
passage_ids = []
passages = []

with open(f'{main_dir}/trec-covid/run.trec-covid.bm25tuned.txt') as f:
  for line in f:
      fields = line.strip().split()
      query_id = fields[0]
      query_ids.append(query_id)
      passage_id = fields[2]
      passage_ids.append(passage_id)

      query_text = id_to_query[query_id]
      queries.append(query_text)

      passage_text = id_to_doc[passage_id]
      passages.append(passage_text)


In [ ]:
len(queries)

50000

In [ ]:
len(passages)

50000

In [ ]:
dataset_test = MSMARCODataset(tokenizer, queries, passages, [1]*len(queries))

In [ ]:
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:

def evaluate_test_dataset2(model, dataloader, set_name):
  scores = []
  model.eval()
  with torch.no_grad():
    for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
      outputs = model(**batch.to(device))
      # Usa os logits brutos
      pos_score = outputs.logits[:,0]
      scores = scores + pos_score.tolist()
  return scores

In [ ]:
logit_scores = evaluate_test_dataset2(model=model, dataloader=dataloader_test, set_name='Test')

Test: 100%|██████████| 1563/1563 [06:44<00:00,  3.86it/s]


In [ ]:
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [ ]:
import pandas as pd

run_trec_file_logits = pd.read_csv(trec_file_logits, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_logits["q0"] = "q0"
run_trec_file_logits = run_trec_file_logits.to_dict(orient="list")

In [ ]:
eval_ndcg10(run_trec_file_logits)

0.5269229736376339

## Finetuning with more datasets

In [ ]:
import json
import glob

# Set the path of the directory containing JSONL files
directory_path = f"{main_dir}/trec-covid/datasets/"
print(directory_path)

# Define an empty list to store the concatenated data
data = []

# Loop through all the JSONL files in the directory
for filename in glob.glob(directory_path + "*.jsonl"):
  print(filename)
  with open(filename, "r") as file:
      # Read each line in the file and append to the data list
      for line in file:
          data.append(json.loads(line))

# Write the concatenated data to a new JSONL file
with open("concatenated.jsonl", "w") as outfile:
    for item in data:
        # Write each item as a JSON object on a separate line
        outfile.write(json.dumps(item) + "\n")


/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/monique_monteiro_1000_queries.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/leandro_carisio_01.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/manoel_2k_generated_queries_20230501.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/leonardo_avila_queries_v1.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/leonardo_pacheco_1k_generated_queries_20230502.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/manoel_1k_generated_queries_20230430.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/marcus_borela_1k_gptj6b_20230501.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/marcus_borela_1k_gptj6b_20230501_v2.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/datasets/mirelle_1k_generated_queries_20230501.jsonl
/content/gdrive/MyDrive/Unicamp-aula-9/trec-covid/da

In [ ]:
!head concatenated.jsonl

{"query": "What is the most suitable protein for a diagnostic approach for Salmonella Enteritidis and why?", "positive_doc_id": "m1cmkkw3", "negative_doc_ids": ["0o3mryu1", "qpq7i1ya", "j5mkparg", "auoo0dm5", "dqfvrerw"]}
{"query": "What is Cryptosporidium parvum and why is it a major cause of disease in both humans and animals?", "positive_doc_id": "ukbl0svm", "negative_doc_ids": ["k3u2nvpe", "gc11fyms", "xoq9qblv", "20o4ufa3", "3huo5nf0"]}
{"query": "What is the role of the renin-angiotensin-aldosterone system in the context of SARS-CoV-2 infection?", "positive_doc_id": "12o4zey2", "negative_doc_ids": ["6gd6nwpu", "dt4t2wos", "8zwfkken", "sv7xpi4f", "6pf73z08"]}
{"query": "What are the functions of individual endolysosomal proteases in cellular processes such as autophagy and lipoprotein particle degradation?", "positive_doc_id": "eqv6a7tj", "negative_doc_ids": ["gmrty2uu", "uzn214j6", "032utjfh", "efet3ozc", "0muwl6oc"]}
{"query": "What is the prevalence of olfactory dysfunction in 

In [ ]:
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"

In [ ]:
X_train, Y_train, X_val, Y_val = generate_training_data("concatenated.jsonl", model_name)

cross-encoder/ms-marco-MiniLM-L-6-v2
                                               query  \
0  What is the most suitable protein for a diagno...   
1  What is Cryptosporidium parvum and why is it a...   
2  What is the role of the renin-angiotensin-aldo...   
3  What are the functions of individual endolysos...   
4  What is the prevalence of olfactory dysfunctio...   

                                             passage  score  
0  Rapid identification of novel antigens of Salm...   True  
1  Cryptosporidium and host resistance: historica...   True  
2  Understanding the Renin-Angiotensin-Aldosteron...   True  
3  Specific functions of lysosomal proteases in e...   True  
4  Olfactory and rhinological evaluations in SARS...   True  


In [ ]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

In [ ]:
len(train_queries)

31808

In [ ]:
len(val_queries)

3536

In [ ]:
dataset_train = MSMARCODataset(tokenizer, train_queries, train_passages, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0

In [ ]:
dataset_val = MSMARCODataset(tokenizer, val_queries, val_passages, Y_val)
assert len(dataset_val[0]['input_ids']) > 0
assert len(dataset_val[1]['attention_mask']) > 0

In [ ]:

dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_val, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [ ]:
model = train(model_name, 10, lr=1e-3)

Parameters 22713601


Valid: 100%|██████████| 111/111 [00:28<00:00,  3.94it/s]


Valid loss: 38.423; Valid accuracy: 0.521


Train: 100%|██████████| 994/994 [08:52<00:00,  1.87it/s]


Epoch: 1 Training loss: 0.67


...


Valid: 100%|██████████| 111/111 [00:25<00:00,  4.33it/s]


Valid loss: 0.017; Valid accuracy: 0.968


Train: 100%|██████████| 994/994 [08:38<00:00,  1.92it/s]


Epoch: 10 Training loss: 0.00



Valid:  72%|███████▏  | 80/111 [00:19<00:06,  4.56it/s]

In [158]:
 model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/best_checkpoint').to(device)

In [159]:
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Valid: 100%|██████████| 111/111 [00:25<00:00,  4.38it/s]

Valid loss: 0.017; Valid accuracy: 0.968


0.9677601809954751

In [160]:
logit_scores = evaluate_test_dataset2(model=model, dataloader=dataloader_test, set_name='Test')

Test: 100%|██████████| 1563/1563 [06:48<00:00,  3.83it/s]


In [164]:
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [165]:
import pandas as pd

run_trec_file_logits = pd.read_csv(trec_file_logits, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_logits["q0"] = "q0"
run_trec_file_logits = run_trec_file_logits.to_dict(orient="list")

In [166]:
eval_ndcg10(run_trec_file_logits)

0.6204472079872368

In [168]:
model = train(model_name, 10, lr=3e-5)

Parameters 22713601


Valid: 100%|██████████| 111/111 [00:27<00:00,  3.97it/s]


Valid loss: 38.423; Valid accuracy: 0.521


Train: 100%|██████████| 994/994 [08:36<00:00,  1.93it/s]


Epoch: 1 Training loss: 2.71


...

Valid loss: 0.031; Valid accuracy: 0.855



Train: 100%|██████████| 994/994 [09:05<00:00,  1.82it/s]


Epoch: 10 Training loss: 0.02



Epochs: 100%|██████████| 10/10 [1:33:05<00:00, 558.54s/it]

Valid loss: 0.028; Valid accuracy: 0.848


In [169]:
 model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/best_checkpoint').to(device)

In [170]:
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Valid: 100%|██████████| 111/111 [00:29<00:00,  3.82it/s]

Valid loss: 0.034; Valid accuracy: 0.887


0.8871606334841629

In [171]:
logit_scores = evaluate_test_dataset2(model=model, dataloader=dataloader_test, set_name='Test')

Test: 100%|██████████| 1563/1563 [06:50<00:00,  3.81it/s]


In [172]:
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [173]:
import pandas as pd

run_trec_file_logits = pd.read_csv(trec_file_logits, sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rank", "score", "system"])
run_trec_file_logits["q0"] = "q0"
run_trec_file_logits = run_trec_file_logits.to_dict(orient="list")

In [174]:
eval_ndcg10(run_trec_file_logits)

0.6751980134653743

## Checking microsoft/MiniLM-L12-H384-uncased in the full dataset

In [19]:
model_name = 'microsoft/MiniLM-L12-H384-uncased'

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [21]:
X_train, Y_train, X_val, Y_val = generate_training_data("concatenated.jsonl", model_name)

Streaming output truncated to the last 5000 lines.
<ipython-input-9-dad32a99c9c2>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0.0},
<ipython-input-9-dad32a99c9c2>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1.0},
<ipython-input-9-dad32a99c9c2>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0.0},
<ipython-input-9-dad32a99c9c2>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row

microsoft/MiniLM-L12-H384-uncased
                                               query  \
0  What is the most suitable protein for a diagno...   
1  What is Cryptosporidium parvum and why is it a...   
2  What is the role of the renin-angiotensin-aldo...   
3  What are the functions of individual endolysos...   
4  What is the prevalence of olfactory dysfunctio...   

                                             passage  score  
0  Rapid identification of novel antigens of Salm...    1.0  
1  Cryptosporidium and host resistance: historica...    1.0  
2  Understanding the Renin-Angiotensin-Aldosteron...    1.0  
3  Specific functions of lysosomal proteases in e...    1.0  
4  Olfactory and rhinological evaluations in SARS...    1.0  


<ipython-input-9-dad32a99c9c2>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0.0},
<ipython-input-9-dad32a99c9c2>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1.0},
<ipython-input-9-dad32a99c9c2>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0.0},
<ipython-input-9-dad32a99c9c2>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1.0},
<ipython-input-9-dad32a99c9c2>:3

In [22]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

train_queries_tokenized = tokenizer(train_queries, truncation=True, max_length=max_length_query)
train_passages_tokenized = tokenizer(train_passages, truncation=True, max_length=max_length_passage)
val_queries_tokenized = tokenizer(val_queries, truncation=True, max_length=max_length_query)
val_passages_tokenized = tokenizer(val_passages, truncation=True, max_length=max_length_passage)

In [23]:
dataset_train = Dataset(train_queries_tokenized, train_passages_tokenized, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0

In [24]:
dataset_valid = Dataset(val_queries_tokenized, val_passages_tokenized, Y_val)

In [25]:
len(dataset_train)

31808

In [26]:
len(dataset_valid)

3536

In [27]:
#dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
#dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)
dataloader_train = data.DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=16, shuffle=False, collate_fn=collate_fn)


for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model = train(model_name, 5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/221 [00:00<?, ?it/s]

Valid loss: 0.694; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train:   0%|          | 0/1988 [00:00<?, ?it/s]